In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_huggingface import HuggingFaceEndpoint

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [6]:
llm = HuggingFaceEndpoint(
        repo_id="meta-llama/Llama-3.1-8B-Instruct",
        temperature=0.5,
        huggingfacehub_api_token=KEY
)

/opt/anaconda3/envs/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#llm

In [9]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [13]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

/var/folders/h3/dshrqqgs1xv628f0c7bj8mj80000gn/T/ipykernel_29727/2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [18]:
file_path="/Users/tharushavihanga/Developer/MCQ-Generator-project/data.txt"

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.

UC Berkeley breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and the model estimate. The alg

In [21]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [23]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

/var/folders/h3/dshrqqgs1xv628f0c7bj8mj80000gn/T/ipykernel_29727/3120652783.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.

UC Berkeley breaks out the learning system of a machine learning algorithm into three main parts.

A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, t

In [24]:
print("Quiz Generated:")
print(response['quiz'])

print("\n" + "=" * 50 + "\n")

print("Review:")
print(response['review'])

Quiz Generated:
### RESPONSE_JSON
{"1": {"mcq": "What is machine learning?", "options": {"a": "A type of artificial intelligence", "b": "A type of biology", "c": "A type of physics", "d": "A type of chemistry"}, "correct": "a"}, "2": {"mcq": "What is the main difference between machine learning and deep learning?", "options": {"a": "Machine learning uses labeled datasets, while deep learning uses unstructured data", "b": "Machine learning uses unstructured data, while deep learning uses labeled datasets", "c": "Machine learning is faster than deep learning", "d": "Machine learning is slower than deep learning"}, "correct": "a"}, "3": {"mcq": "What is a neural network?", "options": {"a": "A type of machine learning algorithm", "b": "A type of deep learning algorithm", "c": "A type of neural network is a sub-field of machine learning", "d": "A neural network is a sub-field of neural networks"}, "correct": "c"}, "4": {"mcq": "What is the main goal of supervised learning?", "options": {"a"

In [25]:
response

{'text': 'Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.\n\nUC Berkeley breaks out the learning system of a machine learning algorithm into three main parts.\n\nA Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.\n\nAn Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.\n\nA Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and the model

In [26]:
# Basic metrics for Hugging Face response (since detailed token tracking isn't available)
quiz_length = len(response['quiz']) if response.get('quiz') else 0
review_length = len(response['review']) if response.get('review') else 0
total_response_length = quiz_length + review_length

print("=== Response Metrics ===")
print(f"Execution Time: {execution_time:.2f} seconds" if 'execution_time' in globals() else "Execution Time: Not available (run cell 21 first)")
print(f"Quiz Response Length: {quiz_length} characters")
print(f"Review Response Length: {review_length} characters") 
print(f"Total Response Length: {total_response_length} characters")
print("\nNote: Hugging Face Hub API doesn't provide detailed token usage like OpenAI")
print("Consider using local Hugging Face models with transformers library for token counting")


=== Response Metrics ===
Execution Time: Not available (run cell 21 first)
Quiz Response Length: 2030 characters
Review Response Length: 2030 characters
Total Response Length: 4060 characters

Note: Hugging Face Hub API doesn't provide detailed token usage like OpenAI
Consider using local Hugging Face models with transformers library for token counting


In [27]:
quiz = response.get('quiz')

In [28]:
# quiz=json.loads(quiz)

In [29]:
import re

# Extract JSON from the quiz response
def extract_json_from_response(text):
	# Find the JSON portion between curly braces
	start_idx = text.find('{')
	if start_idx == -1:
		return None
	
	# Find the matching closing brace
	brace_count = 0
	end_idx = start_idx
	for i, char in enumerate(text[start_idx:], start_idx):
		if char == '{':
			brace_count += 1
		elif char == '}':
			brace_count -= 1
			if brace_count == 0:
				end_idx = i
				break
	
	json_text = text[start_idx:end_idx + 1]
	return json_text

# Extract and parse the JSON
try:
	json_text = extract_json_from_response(quiz)
	if json_text:
		quiz_dict = json.loads(json_text)
		print("Successfully parsed quiz JSON:")
		print(json.dumps(quiz_dict, indent=2))
	else:
		print("No valid JSON found in the quiz response")
		print("Raw quiz content:")
		print(quiz[:500] + "..." if len(quiz) > 500 else quiz)
except json.JSONDecodeError as e:
	print(f"JSON decode error: {e}")
	print("Extracted text:")
	print(json_text if 'json_text' in locals() else "No JSON text extracted")


Successfully parsed quiz JSON:
{
  "1": {
    "mcq": "What is machine learning?",
    "options": {
      "a": "A type of artificial intelligence",
      "b": "A type of biology",
      "c": "A type of physics",
      "d": "A type of chemistry"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "What is the main difference between machine learning and deep learning?",
    "options": {
      "a": "Machine learning uses labeled datasets, while deep learning uses unstructured data",
      "b": "Machine learning uses unstructured data, while deep learning uses labeled datasets",
      "c": "Machine learning is faster than deep learning",
      "d": "Machine learning is slower than deep learning"
    },
    "correct": "a"
  },
  "3": {
    "mcq": "What is a neural network?",
    "options": {
      "a": "A type of machine learning algorithm",
      "b": "A type of deep learning algorithm",
      "c": "A type of neural network is a sub-field of machine learning",
      "d": "A neural network i

In [30]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [31]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A type of artificial intelligence | b: A type of biology | c: A type of physics | d: A type of chemistry',
  'Correct': 'a'},
 {'MCQ': 'What is the main difference between machine learning and deep learning?',
  'Choices': 'a: Machine learning uses labeled datasets, while deep learning uses unstructured data | b: Machine learning uses unstructured data, while deep learning uses labeled datasets | c: Machine learning is faster than deep learning | d: Machine learning is slower than deep learning',
  'Correct': 'a'},
 {'MCQ': 'What is a neural network?',
  'Choices': 'a: A type of machine learning algorithm | b: A type of deep learning algorithm | c: A type of neural network is a sub-field of machine learning | d: A neural network is a sub-field of neural networks',
  'Correct': 'c'},
 {'MCQ': 'What is the main goal of supervised learning?',
  'Choices': 'a: To analyze and cluster unlabeled datasets | b: To predict numerical values |

In [32]:
quiz = pd.DataFrame(quiz_table_data)

In [33]:
quiz.to_csv("ML_quiz.csv", index=False)